# read in pangeo era5 data 
- make into reasonable size data to download & store on d100 hub

In [ ]:
#read in era5 catalog
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/atmosphere.yaml")
#list(cat)
ds  = cat["era5_hourly_reanalysis_single_levels_sa"].to_dask()
ds

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client

gateway = Gateway()
cluster = gateway.new_cluster()
cluster.adapt(minimum=1, maximum=70)
cluster

In [ ]:
#resample to 1 Month
ds_month  = ds.resample(time='1M').mean(keep_attrs=True)

In [ ]:
#resample spatially to 1 deg
ds_month_deg = ds_month.coarsen(latitude=8,longitude=8,boundary="trim").mean(keep_attrs=True)
ds_month_deg

In [ ]:
#print example
tem = ds_month_deg.t2m[0,:,:].load()
tem.plot()

In [ ]:
# the number of GB involved in the reduction
print('total size:',ds.t2m.nbytes/1e9*17)
print(ds_month_deg.t2m.nbytes/1e9)

In [ ]:
#output data one variable at a time
for var in ds_month_deg:
    tem = ds_month_deg[var].load()
    tem.to_netcdf('./../../data/era5_monthly_2deg'+var+'.nc')

In [ ]:
#create filelist
files=[]
[files.append('./../../data/era5_monthly_2deg'+var+'.nc') for var in ds]
print(files)

In [ ]:
#read in data
ds_combined = xr.open_mfdataset(files,parallel=True)
ds_combined

In [ ]:
#output single file
ds_combined.to_netcdf('./../../data/era5_monthly_2deg_pangeo.nc')

In [ ]:
import xarray as xr
ds = xr.open_dataset('./../../data/era5_monthly_2deg_pangeo.nc')
ds

In [ ]:
ds.t2m.mean({'latitude','longitude'}).plot()